<a href="https://colab.research.google.com/github/shashisaini-12/Image-Classification-Using-Transfer-Learning/blob/main/Transfer_Learning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import seaborn as sn

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.models import model_from_json
from tensorflow.keras import Model as model
import tensorflow as tf
device = '/device:GPU:0'
device
  #Zipped dataset path
dataset_path = "/content/drive/MyDrive/DataSets/fruit_image.zip"
#!unzip '/content/drive/MyDrive/DataSets/fruit_image.zip' -d '/content/drive/MyDrive/DataSets'
afer_extraction_dataset = "/content/drive/MyDrive/DataSets/fruits-360-original-size/fruits-360-original-size"
labeling_job = ImageDataGenerator(rescale = 1.0/255)
train_label = labeling_job.flow_from_directory("/content/drive/MyDrive/DataSets/fruits-360-original-size/fruits-360-original-size/Training",
                                               target_size=(260, 260), batch_size=32)
test_label = labeling_job.flow_from_directory("/content/drive/MyDrive/DataSets/fruits-360-original-size/fruits-360-original-size/Test",
                                              target_size=(260, 260), batch_size=32)
validation_label = labeling_job.flow_from_directory("/content/drive/MyDrive/DataSets/fruits-360-original-size/fruits-360-original-size/Validation",
                                                    target_size=(260, 260), batch_size=32)
Model = Sequential()

Model.add(Conv2D(32, (3, 3), input_shape=(260, 260, 3)))
Model.add(MaxPool2D((3, 3)))

Model.add(Conv2D(64, (3, 3)))
Model.add(MaxPool2D(3, 3))

Model.add(Conv2D(128, (3, 3)))
Model.add(MaxPool2D((3, 3)))

Model.add(Conv2D(256, (3, 3)))
Model.add(MaxPool2D((3, 3)))

Model.add(MaxPool2D((2, 2)))
Model.add(Flatten())

Model.add(Dense(train_label.num_classes, activation = 'softmax'))

Found 6231 images belonging to 24 classes.
Found 3110 images belonging to 24 classes.
Found 3114 images belonging to 24 classes.


In [ ]:
Model.compile(optimizer = 'adam', loss='CategoricalCrossentropy', metrics=['accuracy'])

In [ ]:
Model.fit(train_label, epochs =1, validation_data=validation_label)

98/98 [==============================] - ETA: 0s - loss: 0.8136 - accuracy: 0.7551 

In [ ]:
testing_path = "/content/drive/MyDrive/DataSets/fruits-360-original-size/fruits-360-original-size/Test"

In [ ]:
for i in os.listdir(testing_path):
  for j in os.listdir(os.path.join(testing_path, i)):
    my_img = load_img(os.path.join(testing_path, i, j))
    x = np.expand_dims(my_img.resize((260, 260)), axis=0)
    y = Model.predict(x)
    y_class = y.argmax()
    plt.imshow(my_img)
    print('correct class for image {}'.format(i))
    print('predicted class is {}'.format(list(train_label.class_indices.keys())[y_class]))


In [ ]:
y_pred = []
y_actual = []
test_label.reset()

for i in range(len(test_label.filenames)):
  x, y = test_label.next()
  y_pred.append(Model.predict(x).argmax())
  y_actual.append(y.argmax())

In [ ]:
y_actual[:5]

In [ ]:
pred_list = [list(train_label.class_indices.keys())[i] for i in y_pred]
actual_list = [list(train_label.class_indices.keys())[i] for i in y_actual]

In [ ]:
output_df = pd.DataFrame(np.vstack([pred_list, actual_list]).T, columns=['Predicted_class', 'Actual_class'])

In [ ]:
confusion_matrix = pd.crosstab(output_df['Actual_class'], output_df['Predicted_class'], rownames=['Actual'], colnames=['Predicted'])

In [ ]:
sn.heatmap(confusion_matrix, cmap='copper', annot=True, fmt='d')
plt.show

In [ ]:
model_json = Model.to_json()

with open("/content/drive/MyDrive/DataSets/fruits-360-original-size/fruits-360-original-size/saved_model.json", "w") as json_file:
  json_file.write(model_json)

# Serialize weights to HDF5
Model.save_weights("/content/drive/MyDrive/DataSets/fruits-360-original-size/fruits-360-original-size/saved_model.h5")
print("saved model to disk")